<a href="https://colab.research.google.com/github/almemanuel/solve-challenges/blob/main/scripts/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Solve

- Objetivo: obter as sedes de governo, unidades federativas e áreas de uma tabela [desta página](https://pt.wikipedia.org/wiki/Lista_de_capitais_do_Brasil_por_%C3%A1rea)

## Escolha das ferramentas
### API
O Wikipédia possui uma [API](https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:Central_de_pesquisas/Portal_de_dados/API) que facilita a raspagem de dados, visto que o uso de bibliotecas como o Selenium podem trazer particularidades em cada computador. Optei por utilizar o endpoint pois, desta forma, deve funcionar em qualquer computador

### Colab
Acredito que, neste desafio, utilizar notebooks (optei pelo Colab para desenvolver em nuvem) podem fornecer uma visão mais completa da lógica que utilizei, mesclando células de texto e código, além de uma visualização passo a passo mais limpa dos passos que segui

## Extração dos dados

### Bibliotecas necessárias

In [1]:
import pandas as pd
import urllib.request, urllib.parse, json 

### Configurações para consumir a API

#### Obtendo o título da página

O título da página será a principal informação para que possamos obter a tabela com sucesso

In [15]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_capitais_do_Brasil_por_%C3%A1rea'
wiki_title = url.split('/')[-1] # o título é o último elemento na rota
wiki_title

'Lista_de_capitais_do_Brasil_por_%C3%A1rea'

A URL utiliza percent-encode e '_' no lugar de  ' ', por tratar-se de uma rota. Logo, é necessário decodificar este texto

In [16]:
wiki_title = urllib.parse.unquote(wiki_title).replace('_', ' ')
wiki_title

'Lista de capitais do Brasil por área'

Agora, é possível passar a variável `wiki_title` como um dos parametros, mais especificamento o parametro _page_

In [17]:
uri = 'https://pt.wikipedia.org/w/api.php'
params = urllib.parse.urlencode({
    'action': 'parse',
    'prop': 'sections',
    'page': wiki_title,
    'prop': 'text', 
    'format': 'json'
})

Configurando _endpoint_:

In [18]:
from pandas.io.common import urljoin
endpoint = f'{uri}?{params}'

endpoint

'https://pt.wikipedia.org/w/api.php?action=parse&prop=text&page=Lista+de+capitais+do+Brasil+por+%C3%A1rea&format=json'

Consumindo a API

In [24]:
with urllib.request.urlopen(endpoint) as url:
    html = json.loads(url.read().decode())["parse"]["text"]["*"]

Formatando a resposta para manter apenas os dados desejados

In [49]:
df = pd.read_html(html)[0].drop(['Posição', 'Código do IBGE'], axis=1)
df

,Sede de governo,Unidade federativa,Área (km²)
0,Porto Velho,Rondônia,"34 090,952"
1,Manaus,Amazonas,"11 401,092"
2,Rio Branco,Acre,"8 834,942"
3,Campo Grande,Mato Grosso do Sul,"8 082,978"
4,Macapá,Amapá,"6 563,849"
5,Brasília,Distrito Federal,"5 760,783"
6,Boa Vista,Roraima,"5 687,037"
7,Cuiabá,Mato Grosso,"3 266,538"
8,Palmas,Tocantins,"2 227,444"
9,São Paulo,São Paulo,"1 521,110"


## Transformar

### Verificando as colunas

Alguma conversão de tipo pode ser feita?

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Sede de governo     27 non-null     object
 1   Unidade federativa  27 non-null     object
 2   Área (km²)          27 non-null     object
dtypes: object(3)
memory usage: 776.0+ bytes


__Sim!__ A área pode ser convertida em número

### Configurando a área

In [50]:
def remove_space(string):
    return "".join(string.split())

df['Área (km²)'] = df['Área (km²)'].apply(remove_space)
df['Área (km²)'] = df['Área (km²)'].str.replace(',', '.')
df['Área (km²)'] = df['Área (km²)'].astype(float)

df

,Sede de governo,Unidade federativa,Área (km²)
0,Porto Velho,Rondônia,34090.952
1,Manaus,Amazonas,11401.092
2,Rio Branco,Acre,8834.942
3,Campo Grande,Mato Grosso do Sul,8082.978
4,Macapá,Amapá,6563.849
5,Brasília,Distrito Federal,5760.783
6,Boa Vista,Roraima,5687.037
7,Cuiabá,Mato Grosso,3266.538
8,Palmas,Tocantins,2227.444
9,São Paulo,São Paulo,1521.110


### Remoção de notas
Pela fonte ser a Wikipedia, existe a possibilidade de alguns dados conterem referências as notas da página web. Elas são colocadas entre colchetes.
Para que os dados da tabela gerem menos margem de dúvidas, é importante remover essas possíveis notas

In [52]:
def remove_notes(string):
    return string.split('[')[0]

for column in df.columns:
    try:
        df[f'{column}'] = df[f'{column}'].apply(remove_notes)
    except:
        pass # significa que a coluna tem tipo diferente de string

df

,Sede de governo,Unidade federativa,Área (km²)
0,Porto Velho,Rondônia,34090.952
1,Manaus,Amazonas,11401.092
2,Rio Branco,Acre,8834.942
3,Campo Grande,Mato Grosso do Sul,8082.978
4,Macapá,Amapá,6563.849
5,Brasília,Distrito Federal,5760.783
6,Boa Vista,Roraima,5687.037
7,Cuiabá,Mato Grosso,3266.538
8,Palmas,Tocantins,2227.444
9,São Paulo,São Paulo,1521.110
